# Deep Reinforcement Learning Algorithms for Robotic Manipulation
## Key Concepts and Algorithms of Reinforcement Learning

### Markov Decision Process and RL
$$ MDP = (S, A, r, T, \gamma) $$
- $S$ - a set of states
- $A$ - actions
- $r, S \times A \to R$ - the function specifying a reward of taking an action in a state
- $T, S \times A \times S \to R$ - the state transition function
- $\gamma$ - the discount factor implying that a reward obtained in the future is worth a smaller amount than an immediate reward

Solving an MDP involves finding a policy that determines the optimal action for each state, with the goal of maximizing the long-term discounted expected reward.

### Value-Based RL

#### Q-Learning
$$ Q(s, a) = r(s, a) + \gamma \max_{a} Q(s', a) \tag{1} $$

$Q(s, a)$ is the Bellman action-value function, which estimates how good it is to take an action at a given state. Q-Learning is off-policy that learns the optimal policy directly.

#### SARSA
$$ Q(s, a) = Q(s, a) + \alpha \left[ R + \gamma Q(s', a') - Q(s, a) \right] \tag{2}$$
SARSA is on-policy. 

#### Deep Q-Learning (DQN)
DQN uses a neural network to approximate the Q-values. The loss function such as the MSE loss can be:
$$ L = \left(Q(s, a;\theta) - (r + \gamma \max_{a'} Q(s', a';\theta)) \right) ^ 2 \tag{3}$$

The update rule depends on the values produced by the network itself, making convergence diffucult. To address this, the DQN algorithm introduces the use of a replay buffer and target networks. The replay buffer stores past interactions as a list of tuples, which can be sampled to update the value and policy networks. This allows the network to learn from individual tuples multiple times and reduces dependence on the current experience. The target networks are time-delayed copies of the policy and Q-networks, and their parameters are updated according to the following equations:

$$ \theta_Q' \leftarrow \tau \theta_Q + (1 - \tau) \theta_Q' \tag{4}$$
$$ \theta_\mu' \leftarrow \tau \theta_\mu + (1 - \tau) \theta_\mu' \tag{5}$$

where $\theta_\mu'$ and $\theta_Q'$ denote the parameters of the policy and Q-networks, respectively.

![](./images/dqn.webp)

#### Double Deep Q-Learning (Double DQN)
$$ Q(s,a;\theta) = r + \gamma Q(s', argmax_{a'} Q(s', a'; \theta); \theta') \tag{6}$$
The main neural network, $\theta$ determines the best next action $a'$, while the target network is used to evaluate this action and compute its Q-value. This simple change has been shown to reduce overestimations of Q-values in DQN.

#### Dueling Deep Q-Learning (Dueling DQN)
Dueling DQN improves upon traditional DQN by decomposing the Q-values into two separate components
- the value function $V(s)$, which the expected reward for a given state
- the advantage function $A(s, a)$, which reflects the relative advantage of taking a particular action compared to other actions

### Policy-Based RL
The goal of an RL agent using a PG method is to maximize the expected reward, $J(\pi_\theta)=E_{\tau \sim \pi_\theta}[R(\tau)]$, by adjusting the policy parameters $\theta$. A standard approach to finding the optimal policy is to use gradient ascent, in which the policy parameters are updated according to the following rule:
$$ \theta_{t+1} = \theta_t + \alpha \nabla J(\pi_{\theta_t}) \tag{7}$$

This gradient can be further expanded and reformulated as:
$$ \nabla J(\pi_\theta) = E_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla \theta \log \pi_\theta(a_t|s_t) R(\tau) \right] \tag{8}$$

In PG methods, the policy function, which maps states to actions, is learned explicitly and actions are selected without using a value function.

#### Vanilla Policy Gradient (VPG)
In RL, it is often more important to understand the relative advantage of a particular action, rather than its absolute effectiveness.
$$ A_\pi(s, a) = Q_\pi(s, a) - V_\pi(s) \tag{9}$$
- $A_\pi(s, a)$ - advantage function
- $Q_\pi(s, a)$ - action-value function
- $V_\pi(s)$ - state-value function for the policy $\pi$
- $\tau$ - trajectory according to the policy $\pi_\theta$

$$ \nabla J(\pi_\theta) = E_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla \theta \log \pi_\theta(a_t|s_t) A_\pi(s, a) \right] \tag{10}$$

![](./images/vpg.webp)

#### Trust Region Policy Optimization (TRPO)
Trust region policy optimization (TRPO) constrains the optimization of a policy to a trust region. This region is defined as the area in which local approximations of the function are accurate, and the maximum step size is determined within it. The trust region is then iteratively expanded or shrunk based on how well the new approximation performs.

The policy update in TRPO is given by the following optimization problem, which uses the Kullback–Leibler (KL) divergence between the old and new policies as a measure of change:
$$ \nabla J(\pi_\theta) = E_{\tau \sim \pi_\theta} \left[ \nabla \theta \log \pi_\theta(a_t|s_t) A_\pi(s, a) \right] \tag{11}$$
$$ subject to E_{\tau \sim \pi_\theta} \left[ KL[\pi_{\theta_{old}}(\cdot|s_t), \pi_\theta(\cdot|s_t)] \right] \le \delta \tag{12}$$

$\delta$ is the size of the trust region, and the KL divergence between the old and new policies must be less than $\delta$. This optimization problem can be solved using the conjugate gradient method.

#### Proximal Policy Optimization (PPO)
Proximal policy optimization (PPO) is an algorithm that aims to address the overhead issue of TRPO by incorporating the constraint into the objective function as a penalty:
$$ E_{\tau \sim \pi_\theta} \left[ \nabla \theta \log \pi_\theta(a_t|s_t) A_\pi(s, a) \right] \approx \frac{1}{T} \sum_{t=0}^T \nabla \theta \log \pi_\theta(a_t|s_t) A_\pi(s, a) \tag{13}$$

One challenge of PPO is choosing the appropriate value for C. To address this, the algorithm updates C based on the magnitude of the KL divergence. If the KL divergence is too high, C is increased, and, if it is too low, C is decreased. This allows for effective optimization over the course of training.

### Actor–Critic
Actor–critics combine elements of both value-based and policy-based methods. In this approach, the actor, a policy network, proposes an action for a given state, while the critic, a value network, evaluates the action based on the state-action pair. The critic uses the Bellman equation to learn the Q-function, and the actor is updated based on the Q-function to train the policy. This allows the actor–critic approach to take advantage of the strengths of both value-based and policy-based methods.

![](./images/actor-critic.webp)

#### Advantage Actor–Critic (A2C)
In the advantage actor–critic (A2C) algorithm, the critic is trained to estimate the advantage function instead of the Q-function.

#### Asynchronous Advantage Actor–Critic (A3C)
A key feature of A3C is its asynchronous nature, which permits multiple independent agents (networks) with their own weights to interact with different copies of the environment in parallel, allowing for more efficient exploration of the state-action space.

#### Deep Deterministic Policy Gradient (DDPG)
DDPG combines DQN and DPG. It uses two neural networks: a deterministic policy network and a critic (Q) network. We minimize the mean squared error loss between the original Q-value and the updated Q-value:
$$ L = \frac{1}{N} \sum_i (Q_{old} - (r(s, a) + \gamma \max_a Q_{target}(s', a)))^2 \tag{14}$$

Since the policy is deterministic, it suffered from inefficient exploration when the agent was to explore the environment. To improve the DDPG policy, the authors added Ornstein–Uhlenbeck noise to the selected actions during training. However, more recent research implies that uncorrelated, zero-mean Gaussian noise is effective. Figure 7 shows the updated rule of the deep deterministic policy gradient.

![DDPG](./images/ddpg.webp)

#### Others
- Twin Delayed Deep Deterministic Policy Gradients (TD3)
- Soft Actor–Critic (SAC)

## Reward Engineering
### Imitation Learning

### Curriculum Learning

### Hierarchical Reinforcement Learning